In [1]:
from math import gcd
from functools import reduce

# For computing LCM
def lcm(a, b):
    return abs(a * b) // gcd(a, b)

# LCM of multiple numbers
def lcm_multiple(numbers):
    return reduce(lcm, numbers)

# Task class to represent each task
class Task:
    def __init__(self, execution_time, period, m, k):
        self.e = execution_time  # execution time
        self.p = period  # period
        self.m = m  # weakly-hard constraint
        self.k = k  # weakly-hard constraint
        self.jobs = []  # Jobs will be generated later
        self.acceptance_pattern = []  # R-pattern

# Generate jobs based on the period and hyper-period
def generate_jobs(tasks):
    hyper_period = lcm_multiple([task.p for task in tasks])
    for task in tasks:
        num_jobs = hyper_period // task.p
        task.jobs = [(i * task.p, (i + 1) * task.p) for i in range(num_jobs)]
        generate_r_pattern(task)


# R-pattern based on the formula from the image
def generate_r_pattern(task):
    m, k = task.m, task.k
    pattern = []

    # Generate the R-pattern using the original formula logic
    for j in range(k):
        if 0 <= j < m:
            pattern.append(1)  # Accept the first `m` jobs
        else:
            pattern.append(0)  # Reject the remaining `k-m` jobs

    # Extend the pattern to match the number of jobs in the hyper-period
    num_jobs = len(task.jobs)
    task.acceptance_pattern = pattern * (num_jobs // k) + pattern[:num_jobs % k]

# First-fit bin packing algorithm considering e/p instead of just execution time
def first_fit(tasks, processors, demand_function):
    # Sort tasks by increasing order of e/p ratio
    tasks = sorted(tasks, key=lambda task: task.e / task.p)

    for task in tasks:
        assigned = False
        for proc in processors:
            if demand_function(proc, task):
                proc.append(task)
                assigned = True
                break
        if not assigned:
            processors.append([task])

# Demand-based condition for EDF feasibility, considering e/p
def demand_function(processor, task):
    for t1, t2 in task.jobs:
        demand = sum(t.e / t.p for t in processor if t1 <= t.p and t.p <= t2)
        if demand + task.e / task.p > (t2 - t1) / task.p:
            return False
    return True

# R-pattern acceptance check
def check_r_pattern(task):
    # Check sliding windows of size k for R-pattern
    num_jobs = len(task.acceptance_pattern)
    for i in range(num_jobs - task.k + 1):
        window = task.acceptance_pattern[i:i + task.k]
        if sum(window) < task.m:
            return False  # The task doesn't satisfy the (m, k) constraint in this window
    return True

# Function to read tasks from a file
def read_tasks_from_file(filename):
    tasks = []
    with open(filename, 'r') as file:
        for line in file:
            if line.strip():  # Ignore empty lines
                e, p, m, k = map(int, line.split())
                tasks.append(Task(e, p, m, k))
    return tasks

# Main function to schedule tasks and find minimum processors
def schedule_tasks(tasks):
    processors = []  # List of processors
    generate_jobs(tasks)  # Generate jobs for the tasks
    for task in tasks:
        if not check_r_pattern(task):
            print(f"Task with period {task.p} does not satisfy the (m, k) acceptance pattern.")
            return None  # Task doesn't meet (m, k) constraint
    first_fit(tasks, processors, demand_function)  # Use first-fit bin packing (with sorting by e/p)
    return len(processors)  # Return number of processors used

# Read tasks from the input file
tasks = read_tasks_from_file('input.txt')

# Schedule the tasks
min_processors = schedule_tasks(tasks)
if min_processors is not None:
    print("Minimum number of processors required:", min_processors)


Minimum number of processors required: 4
